# SAX Encoding

## Cos'è SAX Encoding?

Il [Symbolic Aggregate approXimation](http://www.cs.ucr.edu/~eamonn/SAX.htm) (SAX) encoding è un metodo per semplificare le serie storiche. E' stato inventato da Eamonn Keogh e Jessica Lin nel 2002.

SAX è un modo di trasformare una serie temporale in una sequenza di simboli. L'idea di base è che ogni simbolo rappresenta un intervallo.

Questa tecnica permette di condurre una *dimensionality reduction* sulle serie storiche quindi possiamo considerarlo un metodo non supervisionato.

Bisogna evidenziare che il SAX è una tecnica molto robusta ai valori mancanti e quindi utilizzabile quando si è in presenza di serie di lunghezza diversa.

> Per rafforzare l'ultima considerazione vi invito a leggere un [articolo di KDNuggets dove viene utilizzato SAX Encoding](https://www.kdnuggets.com/2019/09/time-series-baseball.html) per trovare anomalie tra le stagioni di varie squadre di baseball. Non tutte le stagioni hanno lo stesso numero di match e alcuni dati sono mancanti.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

dati_regioni = pd.read_csv("./dpc-covid19-ita-regioni.csv")

In [ ]:
dati_regioni.head()

## Dataset
Visto il momento in cui viene scritto questo articolo proverò a utilizzare il SAX Encoding sui [dati della protezione civile](https://github.com/pcm-dpc/COVID-19) cercare i pattern tra le serie regionali dei contagi da [*Covid-19*](https://it.wikipedia.org/wiki/COVID-19).

* L'orizzonte temporale và dal 24/02/2020 al 03/04/2020.
* La distribuzione geografica è per Regione anche se il Trentino - Alto Adige è diviso nelle due province autonome.

La colonna che useremo sarà *totale_positivi*.

> Riporto direttamente dal readme del [repository ufficiale della protezione civile](https://github.com/pcm-dpc/COVID-19)
Totale attualmente positivi (ospedalizzati + isolamento domiciliare)

> #### ATTENZIONE!!! IMPORTANTE.
> **Naturalmente non sono un medico/epidemiologo/virologo/... quindi il mio interesse è solo trovare un dataset che mi permettesse di spiegare come funziona l'algoritmo, non trarrò nessuna conclusione dai dati.**

In [ ]:
totale_positivi = dati_regioni[["data", "denominazione_regione", "totale_positivi"]].copy()
totale_positivi.head()

## Cos'è il SAX Encoding?

Il SAX encoding è un metodo per semplificare le serie storiche, viene ridotta la dimensionalità col fine di trovare pattern anomali.

Può essere considerato un metodo non supervisionato di *anomaly detection* dove per anomalia (o outlier) non si intende la singola osservazione ma l'intera serie storica.

Le serie in questo caso sono tutte uguali, visto che non c'è assenza di dati, ma bisogna evidenziare che il SAX è una tecnica molto robusta ai valori mancanti e quindi utilizzabile quando si è in presenza di serie di lunghezza diversa, come nell'articolo sul baseball citato in precedenza.

## Come funziona l'algoritmo

### Step 1: Standardizzare la serie

Per applicare questa tecnica bisogna avere la serie storica sulla riga e ad ogni colonna coinciderà uno step temporale.

In [ ]:
tot_pos_pivot = totale_positivi.pivot(index="data", columns="denominazione_regione", values="totale_positivi").reset_index()

In [ ]:
tot_pos_pivot.head()

In [ ]:
tot_pos_pivot.tail()

In [ ]:
tot_pos_pivot.plot()
#tot_pos_pivot.plot(legend=False)

Le serie devono essere standardizzate così da avere la stessa scala.

$Z = \frac{x - \mu}{\sigma}$

**Questa tecnica non si basa solo sui volumi ma anche sugli andamenti**. In questo caso in numeri assoluti la Lombardia è molto distante dalle altre regioni, ma lo sarà anche come andamento? Questa è la domanda a cui ci aiuterà a rispondere SAX Encoding.

In [ ]:
df_stand = ((tot_pos_pivot.drop("data", axis=1) - tot_pos_pivot.mean())/tot_pos_pivot.std()).T

In [ ]:
df_stand

Ora abbiamo la nostra matrice con sull'indice il nome della serie, ovvero la regione e sulle colonne il periodo da 0 a 39.

### Step 2: Piecewise Aggregate Approximation

Come accennato inzialmente questo metodo riduce la dimensionalità della serie e lo fa proprio con la [P.A.A.](https://vigne.sh/posts/piecewise-aggregate-approx/), l'idea alla base è "*riassumere*" la sequenza in una serie di segmenti che ne riducano la lunghezza ma ne rappresentino ugualmente l'informazione.

Data una time series $Y =$ [$Y_{1}$, $Y_{2}$, ..., $Y_{n}$] che può essere ridotta in una sequenza $X =$ [$X_{1}$, $X_{2}$, ..., $X_{m}$] con $m \leq n$ l'equazione che sintetizza gli elementi della time series originale per adattarli alla nuova dimensione è la seguente:

$\bar{X}_i = \frac{m}{n} \cdot \sum_{j=n/N(i-1) + 1}^{(n/M)\cdot i} x_j$

Quindi la media per ogni finestra temporale che costituirà la nuova sequenza. Si possono notare subito due casi particolari:
* m = n, la nuova sequenza è identica alla serie storica originale
* m = 1, la nuova sequenza ha un solo valore equivalente alla media dei valori dell'intera time series

<br>

![paa](https://vigne.sh/images/PAAStepFinal.png)

<br>

Per applicare la P.A.A. avremo bisogno di scegliere la dimensione della finestra temporale su cui calcolare l'informazione, questo influirà anche su  quanti segmenti verranno utilizzati per ricostruire la serie storica.
> In questo caso visto che i vari ragionamenti (sentiti in TV, ribadisco di non avere nessuna competenza) si basano su un tempo di 2 settimane imposterò il parametro (window) *w=14*.
 
Ridimensionata, ogni serie sarà una sequenza di *lunghezza_serie*/*w* periodi si prende la media del periodo come nuovo valore.

> **Attenzione:** Bisogna impostare un controllo per non perdere informazioni, quindi **se il nuovo numero di periodi non è un intero và sempre arrotondato per eccesso.**

In [ ]:
windows = 14

new_len = int(np.ceil((df_stand.shape[1]/windows)))

# Crea il nuovo dataFrame
df_PAA = pd.DataFrame(index = df_stand.index, columns = range(0, new_len))

In [ ]:
# Calcola la media di ogni window
ind = 0
for i in range(0,df_stand.shape[1], windows):
    avg = df_stand.iloc[:,i:i+windows].mean(axis=1)
    df_PAA[ind] = avg.values
    ind +=1

In [ ]:
df_PAA.head()

### Step 3: SAX

Preparato il dato si può lavorare a ciò che contraddistingue questa tecnica. Adesso il dato numerico verrà converito in stringa. La SAX String è derivazione del numero di livelli in cui si vuole suddividere la serie, ogni livello avrà un'etichetta, queste etichette verranno successivamente concatenate creando la SAX String. La scelta dei livelli influisce sul risultato quindi bisognerebbe sempre farsi affiancare da un esperto di dominio, il consiglio è quello di non creare troppi livelli altrimenti si avrebbe un'eterogeneità troppo alta e nono si troverebbero pattern.
I livelli possono essere fissati oppure calcolati ad ogni periodo, in questo caso si preferisce la seconda opzione. 

Io non conoscendo il fenomeno supporrò 3 livelli ["A", "B", "C"] e suddividerò i valori:
* dal minimo fino al primo quartile
* tra primo e terzo quartile
* superiore al terzo quartile

In questi caso potremmo interpretarli come:
* pochi contagi, ottima situazione
* situazione nella media
* grave emergenza

Una volta definiti i livelli questi vengono concatenati in un'unica stringa detta appunto "*SAX string*". Es. ABAAB

In [ ]:
# Sto ridondando con i DataFrame creati ma è per mostrare i vari step, sorry.
binned = pd.DataFrame(index = df_PAA.index, columns = df_PAA.columns)

for j in range(0, df_PAA.shape[1]):
            bins = []
            bins.append(df_PAA[j].min()-.01)
            bins.append(df_PAA[j].quantile([0.25]).values[0])
            bins.append(df_PAA[j].quantile([0.75]).values[0])
            bins.append(df_PAA[j].max()+.01)
            labels = ["A", "B", "C"]
            binned[j] = pd.cut(df_PAA[j], bins, labels=labels)
binned['sequence'] = binned.apply(''.join, axis=1)
            
binned

### Step 4: Quando un outlier è un outlier?

In questo caso torna indispensabile la presenza di un esperto di dominio, io non essendolo (e lo scriverò ogni volta) ho fatto delle prove fino a trovare un risultato che mi permetta di mostrare qualcosa.

Infatti una volta ottenute le SAX String bisogna contare le frequenze di ognuna e fissare una soglia per cui una serie è considerata anomala.

In [ ]:
# imposto il limite a 1
limit = 1

# serie originale trasposta
encoded = tot_pos_pivot.T
encoded.drop(index="data", inplace = True)

# aggiungo l'etichetta se è un outlier o no
freq = binned.sequence.value_counts()
encoded['outlier'] = binned['sequence'].isin(list(freq[freq<=limit].index))

In [ ]:
encoded.head()

In [ ]:
encoded["outlier"].value_counts()

Da questo esperimento, considerando una finestra di 14 giorni e che un outlier per noi è chi ha una sequenza unica (viste anche le sole 21 serie) abbiamo 6 regioni che hanno lo stesso andamento e 15 che si discostano da tutte le altre.

In [ ]:
plt.figure(figsize=(18,10))
for i in range(encoded.shape[0]):
    if encoded.iloc[i,-1]:
        col = 'r'
    else:
        col = 'b'
    plt.plot(encoded.iloc[i,:-1], col)
plt.legend()
plt.title("Totale positivi per Regione dopo SAX Encoding")

Ultima cosa da notare è che non sono solo i volumi ad influenzare questa tecnica ma anche gli andamenti come testimoniano le serie di Friuli e Lombardia che hanno volumi molto differenti ma probabilmente andamenti simili.

*Se volete provare altre soluzioni potete provare a variare la finestra temporale e la soglia per essere considerato outlier*.

## Come utilizzare le SAX String?

### Anomaly Detection

Il SAX Encoding può essere considerato un metodo non supervisionato di *anomaly detection* dove per anomalia (o outlier) non si intende la singola osservazione ma l'intera serie storica.

Possiamo calcolare la frequenza di ogni SAX String, impostato il limite che per noi determina  un outlier etichettare come tale tutte le serie con frequenza della SAX String minore o uguale al bound.

> Nell'immagine ho impostato come bound 1

In rosso abbiamo gli outlier, blu le sequenze con frequenza maggiore di 1, vediamo che quasi ogni regione ha un suo andamento, questo dovuto a diversi fattori (territorio, controlli, ...)

<br>

![Anomaly_new](https://i.imgur.com/Idsost8.png)

<br>

### Clustering

Sempre rimanendo nel campo dei modelli non supervisionati le SAX String possono essere utilizzate per il clustering.

Un'applicazione che mi viene in mente è il clustering gerarchico utilizzando come misura di similarità (o dissimilarità a seconda se agglomerativo o divisivo) la [distanza di Levenshtein](https://it.wikipedia.org/wiki/Distanza_di_Levenshtein).


### Classification

Passando ai metodi supervisionati, avendo già delle serie storiche etichettate, penso ad esempio a quelle sul consumo energetico che hanno anche una certa stagionalità, si potrebbe applicare il KNN alla sequenza per classificare le nuove serie.

### Stream Data

Nel precedente paragrafo ho citato la stagionalità perchè in questo articolo non è stato trattato un tema di forte interesse come il **dato in stream**.
Con la diffusione dell'IoT le time series in stream sono un tema sempre più caldo sia per quanto riguarda l'immagazzinamento dei dati che per le analisi.

Nel caso di serie con stagionalità, o forti componenti cicliche, oltre alla finestra temporale per la P.A.A. si potrebbe pensare ad una finestra per l'aggiornamento dei periodi con lo stesso timing dell'effetto. Ad esempio, se ho una serie con ciclicità mensile e voglio valutare i pattern su finestre settimanali potrei impostare:
* orizzonte di aggiornamento mensile
* finestra per il SAX Encoding settimanale

così da studiare SAX String di lunghezza 4 o 5.

Naturalmente con fenomeni fortemente erratici questo è più complicato da decidere, quasi sicuramente i risultati dei nostri modelli (anomaly detection, classificazione, clustering, ...) varierebbero molto frequentemente.

Prendendo ad esempio il nostro dataset. Se ci fermassimo una settimana prima, nel grafico seguente i dati si fermano al 13 aprile, le sequenze non singleton sarebbero molte di più. Questo solo per rimarcare ancora una volta l'importanza della scelta dela giusta finestra temporale e del periodo di osservazione in relazione al fenomeno che si và ad analizzare.

<br>

![Anomaly](https://i.imgur.com/8VrDJaq.png)

<br>

> Se volete approfondire questo tema vi consiglio il paper ["*A Symbolic Representation of Time Series, with Implications for
Streaming Algorithms*" di Jessica Lin, Eamonn Keogh, Stefano Lonardi, Bill Chiu](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=4&cad=rja&uact=8&ved=2ahUKEwipqu-n6PnoAhUPY8AKHZfhCVwQFjADegQIAxAB&url=http%3A%2F%2Fwww.cs.ucr.edu%2F~eamonn%2FSAX.pdf&usg=AOvVaw1Fq0YZQeLmB1WXQFoVZFJw).